This notebook demonstrates the conversion of AEM data from aseg-gdf format to netCDF format

Neil Symington
neil.symington@ga.gov.au

In [1]:
%matplotlib inline

from geophys_utils.netcdf_converter import aseg_gdf2netcdf_converter
from geophys_utils.netcdf_converter.aseg_gdf_utils import aseg_gdf_format2dtype
from scipy.io import loadmat
import netCDF4
import os, math
import numpy as np
import matplotlib.pyplot as plt
# SO we can see the logging. This enables us to debug
import gc
import pandas as pd
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
import importlib

DEBUG:root:test
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [9]:
def extract_pmap_info(D):
    """
    @param: D: python dictionary from loadmat function in scipy
    
    returns
    a more usebal dictionary
    """

    freq = D['M']['f'][0,0]


    # Get the changepoint histogram

    cp = D['M']['cp'][0,0].flatten()

    cond_cells = np.linspace(D['M']['vmin'][0,0][0,0], D['M']['vmax'][0,0][0,0], D['M']['nvcells'][0,0][0,0])
    
    laybins = D['M']['lhist'][0,0][0,0][1].flatten()
    lay_prob = D['M']['lhist'][0,0][0,0][2].flatten()

    nsample = D['M']['nsample'][0,0][0,0]

    ndata = int(D['M']['ndata'][0,0][0,0])
    nsamples = np.int(D['M']['nsample'][0,0][0,0])
    
    misfit = []
    sample_no = []
   
    for i in range(nchains):
        misfit.append(D['M']['conv'][0,0]['misfit'][0,i].flatten())
        sample_no.append(D['M']['conv'][0,0]['sample'][0,i].flatten())
        
    
    return {'conductivity_pmap': freq, "change_point_pmap": cp, 'nlayer_bins': laybins, 
            'nlayer_prob': lay_prob, 'nsamples': nsample, 'ndata': ndata, 'misfit': np.array(misfit),
           'sample_no': np.array(sample_no), 'ndata': ndata}

In [3]:

root = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\combined"
nc_out_path = os.path.join(root, "HE_rjmcmc_.nc")

dat_in_path = os.path.join(root, 'rjmcmc.dat')


dfn_in_path = os.path.join(root, 'rjmcmc.dfn')


crs_string = "EPSG:28352"

In [4]:
if os.path.exists(nc_out_path):
    os.remove(nc_out_path)

d2n = aseg_gdf2netcdf_converter.ASEGGDF2NetCDFConverter(nc_out_path, 
                                                 dat_in_path, 
                                                 dfn_in_path,
                                                 crs_string,
                                                 fix_precision=True,
                                                 remove_null_columns = False)
d2n.convert2netcdf()                                      

INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:Reading definitions file C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\combined\rjmcmc.dfn
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:Reading data file C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\combined\rjmcmc.dat


[{'short_name': 'comments', 'format': 'A4', 'long_name': 'COMMENTS', 'dtype': '<U4', 'columns': 1, 'width_specifier': 4, 'decimal_places': 0, 'variable_attributes': {'aseg_gdf_format': 'A4'}}, {'short_name': 'uniqueid', 'format': 'I12', 'long_name': 'Inversion sequence number', 'dtype': 'int64', 'columns': 1, 'width_specifier': 12, 'decimal_places': 0, 'variable_attributes': {'aseg_gdf_format': 'I12'}}, {'short_name': 'survey', 'format': 'I12', 'long_name': 'Survey number', 'dtype': 'int64', 'columns': 1, 'width_specifier': 12, 'decimal_places': 0, 'variable_attributes': {'aseg_gdf_format': 'I12'}}, {'short_name': 'date', 'format': 'I12', 'long_name': 'Date number', 'dtype': 'int64', 'columns': 1, 'width_specifier': 12, 'decimal_places': 0, 'variable_attributes': {'aseg_gdf_format': 'I12'}}, {'short_name': 'flight', 'format': 'I12', 'long_name': 'Flight number', 'dtype': 'int64', 'columns': 1, 'width_specifier': 12, 'decimal_places': 0, 'variable_attributes': {'aseg_gdf_format': 'I12'}

INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:A total of 4299 points were read
c:\users\pcuser\onedrive\github\geophys_utils\geophys_utils\netcdf_converter\aseg_gdf_utils.py:29: RuntimeWarning: invalid value encountered in log10
  fexp[f != 0] = np.ceil(np.log10(np.abs(f[f != 0]))).astype('int32')
c:\users\pcuser\onedrive\github\geophys_utils\geophys_utils\netcdf_converter\aseg_gdf_utils.py:34: RuntimeWarning: divide by zero encountered in true_divide
  fman = f/10.0**fexp
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:Datatype for variable uniqueid changed from int64 to int16
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:Datatype for variable survey changed from int64 to int16
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:Datatype for variable date changed from int64 to int32
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:Datatype for variable flight changed from int64 to int32
INFO:geophys_utils.netcdf_conv

point 4299
layer 150


INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D float32 variable conductivity_mean
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D float32 variable conductivity_mode
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D float32 variable conductivity_p50
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D float32 variable conductivity_p10
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D float32 variable conductivity_p90
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D float32 variable conductivity__highestlikelihood
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D float32 variable conductivity_lowestmisfit
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:	Writing 2D int16 variable change_point
INFO:geophys_utils.netcdf_converter.aseg_gdf2netcdf_converter:Creating crs, longitude and latitude variables for unp

In [5]:
# Create a python object with the EM dataset
d = netCDF4.Dataset(nc_out_path, "a")

In [6]:
# Now we want to add the probability map data

# open a csv that maps the fiducials to their pmap matla files

df = pd.read_csv(r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\combined\rjmcmc_map.csv")

# Makew sure they are the same length
assert len(d['fiducial'][:]) == len(df)


In [10]:
ndpethcells = df['ndepthcells '].iloc[0]
nsamples = len(df)

ncond_cells = 100
nchains = d['nchains'][0].data
# From the matlab files
nmisfit = 1019
min_layers = 1
max_layers = 25
condmin = 0.002
condmax = 3.


# TODO add an assertion function here


In [18]:
# now we create some arrays that will be added to the netCDF file as variables
misfit_count = np.arange(1, nchains + 1)
layer_bins = np.arange(min_layers, max_layers + 1)

pmap_conductivities = np.zeros(shape = (nsamples, ndpethcells, ncond_cells), dtype = np.int32)
maxlayer_counts = np.zeros(shape = (nsamples, layer_bins.shape[0]), dtype = np.int32)
cond_cells = np.linspace(np.log10(condmin), np.log10(condmax), ncond_cells)
misfit= np.zeros(shape = (nsamples, nchains, nmisfit), dtype = np.float64)
sample_no = np.zeros(shape = (nsamples, nchains, nmisfit), dtype = np.int32)
ndata = np.zeros(shape = (nsamples), dtype = np.int)

In [22]:
# Now we need to populate the arrays

# We iterate through the fiducials to ensure we have everything ordered correctly

for i in range(nsamples):
    pmap_file = df[df['fiducial '] == d['fiducial'][i]]['matfile'].values[0]
    D = extract_pmap_info(loadmat(pmap_file))
    # Write data to arrays
    pmap_conductivities[i] = D['conductivity_pmap']
    maxlayer_counts[i] = D['nlayer_prob']
    misfit[i] = D['misfit']
    sample_no = D['sample_no']
    ndata[i] = D['ndata']
    
    

In [25]:
for j in range(sample_no.shape[1]):
    assert np.unique(sample_no[:,j]).shape[0] == 1
    
rj_number = sample_no[0]

In [27]:
# Create new dimensions

cond_pmap_dim = d.createDimension("conductivity_cells", cond_cells.shape[0])

layer_pmap_dim = d.createDimension("nlayers_cells", layer_bins.shape[0])

# Create variables for the associated values for these dimensions

cond_pmap_dim = d.createVariable("conductivity_cells","f8",("conductivity_cells",))

layer_pmap_dim = d.createVariable("nlayers_cells","i4",("nlayers_cells",))

# Fill

cond_pmap_dim[:] = cond_cells
layer_pmap_dim[:] = layer_bins

RuntimeError: NetCDF: String match to name in use

In [28]:
# Create new dimensions

chain_dim = d.createDimension("chain_dim", misfit_count.shape[0])

rjsample_dim = d.createDimension("rj_sample_dim", misfit.shape[2])

# Create variables for the associated values for these dimensions

chain_dim = d.createVariable("chain_no","i4",("chain_dim",))

rjsample_dim = d.createVariable("rj_sample_number","i4",("rj_sample_dim",))

# Fill

chain_dim[:] = misfit_count
rjsample_dim[:] = rj_number

In [29]:
# Now add the remaining variables

cond_pmap =  d.createVariable("conducitivty_bin_count","f8",("point", "layer",
                                                                  "conductivity_cells"))

layer_pmap =  d.createVariable("nlayer_bin_count","f8",("point", "nlayers_cells"))

misfits =  d.createVariable("misfit","f8",("point", "chain_dim", "rj_sample_dim"))


In [30]:
if np.unique(ndata).shape[0] == 1:
    ndat = d.createVariable('n_data', "i4")
    ndat[:] =  np.unique(ndata)[0]
else:
    ndat = d.createVariable('n_data', "i4", ('point',))
    ndat[:] =  ndata

In [31]:
cond_pmap[:] = pmap_conductivities

layer_pmap[:] = maxlayer_counts

misfits[:] = misfit

In [32]:
d.close()

In [28]:
d = netCDF4.Dataset(nc_out_path, "r")

In [29]:
d

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Dataset read from ASEG-GDF file rjmcmc.dat
    Conventions: CF-1.6,ACDD-1.3
    featureType: trajectory
    geospatial_vertical_min: -20.9
    geospatial_vertical_max: 34.3
    geospatial_vertical_units: m
    geospatial_vertical_resolution: point
    geospatial_vertical_positive: up
    history: Converted from ASEG-GDF file C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\combined\rjmcmc.dat using definitions file C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\combined\rjmcmc.dfn
    date_created: 2019-12-23T14:27:39.021963
    geospatial_east_resolution: point
    geospatial_north_resolution: point
    geospatial_east_min: 730488.1875
    geospatial_east_max: 740655.1875
    geospatial_east_units: m
    geospatial_north_min: 8613396.0
    geospatial_north_max: 8623772.0
    geospatial_north_units: m
    geospatial_bounds: POLYGON((730488.1875 8613396.0000, 735163.1250 8623566.